In [4]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import cv2  
import tensorflow as tf
from PIL import Image
from google.colab import drive
from google.colab.patches import cv2_imshow
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle
import os
from sklearn.preprocessing import MinMaxScaler

In [5]:
drive.mount('/content/drive', force_remount=True) 
path = '/content/drive/My Drive/Colab Notebooks/491/' 

Mounted at /content/drive


#LIE

In [ ]:
!pip uninstall scikit-learn
!pip install scikit-learn==0.20.4

In [ ]:
'''
from sklearn import tree
from sklearn import svm
from sklearn import datasets
from sklearn.tree import tree
from sklearn.externals import joblib
loaded_model = joblib.load(path + 'DT_ML_model(Microexpressions).pkl' )
from scipy.io import wavfile

samplerate, data = wavfile.read(path + 'trial_lie_001.mp4')

loaded_model.predict(data)
'''

In [ ]:
!git clone https://github.com/tyiannak/pyAudioAnalysis.git
!pip install -r ./requirements.txt
!pip install -e 

In [ ]:
from sklearn import svm
from sklearn import datasets
import librosa
import librosa.display

In [2]:
def mp3tomfcc(file_path, max_pad):
  audio, sample_rate = librosa.core.load(file_path)
  mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=20)
  pad_width = max_pad - mfcc.shape[1]
  if (pad_width > 0):
    mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
  else:
    mfcc = mfcc[:,0:max_pad]
  return mfcc

In [6]:
mfccs = []
labels = [] # 0 = Truth, 1 = Lie 
file_path = path + 'data/'
index = 0

for f in os.listdir(file_path):
  if f.endswith('.wav'):
      mfccs.append(mp3tomfcc(file_path + '/' + f, 1000)) 
      if "lie" in f:
        labels.append(0)
      elif "truth" in f:  
        labels.append(1)
      index = index + 1
      print(index)
      print(f)

1
trial_lie_040.mp4.wav
2
trial_lie_055.mp4.wav
3
trial_lie_042.mp4.wav
4
trial_lie_043.mp4.wav
5
trial_lie_054.mp4.wav
6
trial_lie_041.mp4.wav
7
trial_lie_056.mp4.wav
8
trial_lie_047.mp4.wav
9
trial_lie_051.mp4.wav
10
trial_lie_046.mp4.wav
11
trial_lie_044.mp4.wav
12
trial_lie_050.mp4.wav
13
trial_lie_053.mp4.wav
14
trial_lie_045.mp4.wav
15
trial_lie_057.mp4.wav
16
trial_lie_022.mp4.wav
17
trial_lie_052.mp4.wav
18
trial_lie_036.mp4.wav
19
trial_lie_021.mp4.wav
20
trial_lie_023.mp4.wav
21
trial_lie_037.mp4.wav
22
trial_lie_009.mp4.wav
23
trial_lie_008.mp4.wav
24
trial_lie_035.mp4.wav
25
trial_lie_034.mp4.wav
26
trial_lie_024.mp4.wav
27
trial_lie_020.mp4.wav
28
trial_lie_031.mp4.wav
29
trial_lie_030.mp4.wav
30
trial_lie_018.mp4.wav
31
trial_lie_025.mp4.wav
32
trial_lie_027.mp4.wav
33
trial_lie_017.mp4.wav
34
trial_lie_032.mp4.wav
35
trial_lie_019.mp4.wav
36
trial_lie_003.mp4.wav
37
trial_lie_033.mp4.wav
38
trial_lie_026.mp4.wav
39
trial_lie_016.mp4.wav
40
trial_lie_015.mp4.wav
41
trial_

In [7]:
mfccs = np.asarray(mfccs)
print(mfccs.shape)
np.save(path+'mfccs_all_loy_no_pad1000.npy', mfccs)
mfccs_loaded = np.load(path+'mfccs_all_loy_no_pad1000.npy')

(121, 20, 1000)


In [8]:
print(labels)
labels = np.asarray(labels)
np.save(path+'labels.npy', labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [9]:
from sklearn.utils import shuffle
mfccs, labels = shuffle(mfccs, labels)

In [10]:
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels, num_classes=None)
print(labels.shape)

(121, 2)


In [11]:
import tensorflow as tf
import IPython.display as ipd
import keras
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing import sequence
from keras.utils import np_utils
from tensorflow.keras.optimizers import Adam

In [12]:
def get_seq_model(input_shape, num_classes):
  model = Sequential()
  model.add(LSTM(units=16, dropout=0.05, recurrent_dropout=0.35, return_sequences=True, input_shape=input_shape))
  model.add(BatchNormalization())
  model.add(LSTM(units=8, dropout=0.05, recurrent_dropout=0.35, return_sequences=True))
  model.add(Flatten())
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001,decay=0.0), metrics=['accuracy'])
  return model

In [13]:
dim_1 = mfccs.shape[1]
dim_2 = mfccs.shape[2]
channels = 1
classes = 2
X = mfccs
y = labels
input_shape = (X.shape[1], X.shape[2])

In [14]:
from sklearn.linear_model import LogisticRegression # Regression classifier
from sklearn.tree import DecisionTreeClassifier # Decision Tree classifier
from sklearn import svm # Support Vector Machine
from sklearn.linear_model import SGDClassifier # Stochastic Gradient Descent Classifier
#from sklearn.ensemble import RandomForestClassifier # Random Forest and Gradient Boosting Classifier
#from sklearn.naive_bayes import MultinomialNB # Naive Bayes Classifier 
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix # Some metrics to check the performance of the models
from sklearn.model_selection import train_test_split
import pickle

In [15]:
Classifiers = {'LR':LogisticRegression(random_state=10,C=5,max_iter=200),
               'DTC':DecisionTreeClassifier(random_state=10,min_samples_leaf=2),
               #'RF':RandomForestClassifier(random_state=10,n_estimators=100,n_jobs=-1),
               #'GBC':GradientBoostingClassifier(random_state=10,n_estimators=400,learning_rate=0.2),
               'SGD':SGDClassifier(loss="hinge", penalty="l2"),
               'SVM':svm.SVC(kernel='linear', C=0.1, probability=True)}

In [16]:
nsamples, nx, ny = X.shape
X_final = X.reshape((nsamples,nx*ny))
y_final = np.argmax(y, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=1)
print(X_train.shape)
print(y_train.shape)

(84, 20000)
(84,)


In [17]:
def print_stats(y_test, pred):
    CM = confusion_matrix(y_test,pred)
    TN, FP, FN, TP = confusion_matrix(y_test, pred).ravel()

    Accuracy = accuracy_score(y_test,pred)
    Precision = TP / (TP + FP)
    Recall = TP / (TP + FN)
    F1 = 2 * (Precision * Recall) / (Precision + Recall)

    print('==='*20)
    print('Accuracy = '+str(Accuracy))
    print('==='*20) 
    print('Precision = '+str(Precision))
    print('==='*20) 
    print('Recall = '+str(Recall))
    print('==='*20) 
    print('F1 = '+str(F1))
    print('==='*20) 
    print(CM)

In [18]:
def ML_Pipeline(clf_name):
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train, y_train)

    #filename = 'LR_model.pkl'
    #pickle.dump(clf, open(path+'models/'+filename, 'wb'))

    predi = clf.predict(X_test)
    
    np.asarray(predi)
    #np.save('MFCC_'+clf_name, predi)
    print_stats(y_test, predi)

In [25]:
ML_Pipeline('SGD')

Accuracy = 0.6216216216216216
Precision = 0.6666666666666666
Recall = 0.13333333333333333
F1 = 0.2222222222222222
[[21  1]
 [13  2]]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/base.py:283: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)


In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 20, 16)            65088     
                                                                 
 batch_normalization (BatchN  (None, 20, 16)           64        
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 20, 8)             800       
                                                                 
 flatten (Flatten)           (None, 160)               0         
                                                                 
 dense (Dense)               (None, 2)                 322       
                                                                 
Total params: 66,274
Trainable params: 66,242
Non-trainable params: 32
___________________________________________________

In [26]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.3, random_state=1)
model = get_seq_model(input_shape, classes)
print(X_train2.shape)
print(y_train2.shape)

history = model.fit(X_train2, y_train2, batch_size=20, epochs=15, verbose=1, validation_split=0.2)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


(84, 20, 1000)
(84, 2)
Epoch 1/15
4/4 [==============================] - 7s 266ms/step - loss: 0.6897 - accuracy: 0.5075 - val_loss: 0.7070 - val_accuracy: 0.5294
Epoch 2/15
4/4 [==============================] - 0s 70ms/step - loss: 0.6651 - accuracy: 0.5970 - val_loss: 0.6996 - val_accuracy: 0.4706
Epoch 3/15
4/4 [==============================] - 0s 69ms/step - loss: 0.6212 - accuracy: 0.8209 - val_loss: 0.7021 - val_accuracy: 0.4706
Epoch 4/15
4/4 [==============================] - 0s 69ms/step - loss: 0.5929 - accuracy: 0.8955 - val_loss: 0.7009 - val_accuracy: 0.4706
Epoch 5/15
4/4 [==============================] - 0s 64ms/step - loss: 0.5897 - accuracy: 0.8209 - val_loss: 0.7102 - val_accuracy: 0.5294
Epoch 6/15
4/4 [==============================] - 0s 64ms/step - loss: 0.5559 - accuracy: 0.8507 - val_loss: 0.7083 - val_accuracy: 0.5294
Epoch 7/15
4/4 [==============================] - 0s 65ms/step - loss: 0.5253 - accuracy: 0.9254 - val_loss: 0.7080 - val_accuracy: 0.5294
Epo

In [ ]:
filename = 'NN_model.pkl'
pickle.dump(history, open(path+'models/'+filename, 'wb'))

INFO:tensorflow:Assets written to: ram://e30b9292-cfdc-47bc-9a43-dd661e1a1e49/assets


INFO:tensorflow:Assets written to: ram://e30b9292-cfdc-47bc-9a43-dd661e1a1e49/assets


In [28]:
print(model.metrics_names)
#model.evaluate(X_train2, y_train2, batch_size = 3, verbose = 1)
model.evaluate(X_test2, y_test2, batch_size = 3, verbose = 1)


['loss', 'accuracy']
13/13 [==============================] - 0s 8ms/step - loss: 0.6086 - accuracy: 0.6757


[0.6085554361343384, 0.6756756901741028]

In [29]:
pred = model.predict(X_test2)
pred_int = np.zeros(pred.shape[0])
y_test2_int = np.zeros( y_test2.shape[0])
cnt = 0
for i in pred:
  if i[0] > i[1]:
    pred_int[cnt] = 0
  else:
    pred_int[cnt] = 1
  cnt += 1 
cnt = 0
for i in y_test2:
  if i[0] > i[1]:
    y_test2_int[cnt] = 0
  else:
    y_test2_int[cnt] = 1
  cnt += 1  
print(y_test2_int)    
print_stats(y_test2_int, pred_int)

[1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1.
 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0.]
Accuracy = 0.6756756756756757
Precision = 0.6
Recall = 0.6
F1 = 0.6
[[16  6]
 [ 6  9]]
